In [130]:
#loading necessaries library 
import pandas as pd
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import ModelSpec as MS, summarize, poly
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm
from scipy.stats import bootstrap

> Question 1

In [131]:
#Loading the data
red_w = pd.read_csv('C:/Users/choih/OneDrive/Desktop/winequality-red.csv' , sep=';')
white_w = pd.read_csv('C:/Users/choih/OneDrive/Desktop/winequality-white.csv' , sep=';')

> Question 2

In [132]:
#check if there are missing value in red_w
red_m = red_w['volatile acidity'].isna()
red_m

0       False
1       False
2       False
3       False
4       False
        ...  
1594    False
1595    False
1596    False
1597    False
1598    False
Name: volatile acidity, Length: 1599, dtype: bool

In [133]:
#check if there are missing value in white_w
white_m = white_w['volatile acidity'].isna()
white_m

0       False
1       False
2       False
3       False
4       False
        ...  
4893    False
4894    False
4895    False
4896    False
4897    False
Name: volatile acidity, Length: 4898, dtype: bool

> Question 3

In [134]:
#build bootstrap sample for red wine dataset
def drawb_sample(D, n=None):
    n = n or D.shape[0]  # Use the number of rows in D if n is not provided
    idx = np.random.choice(len(D), size=n, replace=True)  # Draw bootstrap sample with replacement
    X_star = D.iloc[idx]  # Select the rows from the sampled indices
    return X_star

# build a bootstrap sample (with replacement) from the red_w
bootstrap_s_red = drawb_sample(red_w)

bootstrap_s_red

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1003,6.8,0.360,0.32,1.8,0.067,4.0,8.0,0.99280,3.36,0.55,12.8,7
109,8.1,0.785,0.52,2.0,0.122,37.0,153.0,0.99690,3.21,0.69,9.3,5
867,6.9,0.510,0.23,2.0,0.072,13.0,22.0,0.99438,3.40,0.84,11.2,6
1153,9.6,0.420,0.35,2.1,0.083,17.0,38.0,0.99622,3.23,0.66,11.1,6
320,9.8,0.660,0.39,3.2,0.083,21.0,59.0,0.99890,3.37,0.71,11.5,7
...,...,...,...,...,...,...,...,...,...,...,...,...
495,10.7,0.350,0.53,2.6,0.070,5.0,16.0,0.99720,3.15,0.65,11.0,8
849,6.4,0.630,0.21,1.6,0.080,12.0,32.0,0.99689,3.58,0.66,9.8,5
37,8.1,0.380,0.28,2.1,0.066,13.0,30.0,0.99680,3.23,0.73,9.7,7
1465,6.8,0.590,0.10,1.7,0.063,34.0,53.0,0.99580,3.41,0.67,9.7,5


> Question 4

In [135]:
#build bootstrap sample for white wine dataset
def drawb_sample(D, n=None):
    n = n or D.shape[0]  # Use the number of rows in D if n is not provided
    idx = np.random.choice(len(D), size=n, replace=True)  # Draw bootstrap sample with replacement
    X_star = D.iloc[idx]  # Select the rows from the sampled indices
    return X_star

# build a bootstrap sample (with replacement) from the white_w
bootstrap_s_white = drawb_sample(white_w)

bootstrap_s_white

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
3327,5.8,0.17,0.36,1.3,0.036,11.0,70.0,0.99202,3.43,0.68,10.400000,7
1671,7.0,0.35,0.30,6.5,0.028,27.0,87.0,0.99360,3.40,0.42,11.400000,7
218,5.7,0.28,0.24,17.5,0.044,60.0,167.0,0.99890,3.31,0.44,9.400000,5
3638,6.4,0.29,0.32,2.4,0.014,34.0,89.0,0.99008,3.24,0.66,12.500000,7
464,7.2,0.39,0.62,11.0,0.047,66.0,178.0,0.99760,3.16,0.50,8.700000,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1918,6.6,0.35,0.29,14.4,0.044,54.0,177.0,0.99910,3.17,0.58,8.900000,6
697,7.5,0.31,0.51,14.8,0.039,62.0,204.0,0.99820,3.06,0.60,9.500000,5
2520,7.5,0.41,0.23,14.8,0.054,28.0,174.0,0.99898,3.18,0.49,9.700000,5
724,7.8,0.30,0.40,1.8,0.028,23.0,122.0,0.99140,3.14,0.39,10.900000,7


> Question 5

In [136]:
#build the function for calculating bootstrap median 
def calculateb_median(D, column_name):
    medians = [] #create empty list 
    bootstrap_sample = drawb_sample(D) #call bootstrap sample function
    median_value = np.median(bootstrap_sample[column_name]) #computes the median of the specified column
    medians.append(median_value) #Adds the computed median to the list 
    return np.array(medians) #Converts the list into a NumPy array and returns it.

#Apply dataset into function 
red_medians = calculateb_median(red_w, column_name='volatile acidity')
white_medians = calculateb_median(white_w, column_name='volatile acidity')

print(red_medians)
print(white_medians)


[0.52]
[0.26]


> Question 6

In [137]:
#build bootstrap sample for dataset
def drawb_sample(D, n=None):
    n = n or D.shape[0]  # Use the number of rows in D if n is not provided
    idx = np.random.choice(len(D), size=n, replace=True)  # Draw bootstrap sample with replacement
    X_star = D.iloc[idx]  # Select the rows from the sampled indices
    return X_star

#build the function for calculating bootstrap median that repeats 1000
def calculateb_median(D, column_name, B=1000):
    medians = [] #create empty list 
    for _ in range(B): #for repeats
        bootstrap_sample = drawb_sample(D) #call bootstrap sample function
        median_value = np.median(bootstrap_sample[column_name]) #Adds the computed median to the list 
        medians.append(median_value) #Adds the computed median to the list 
    return np.array(medians) #Converts the list into a NumPy array and returns it

#Apply dataset into function 
red_medians = calculateb_median(red_w, column_name='volatile acidity', B=1000)
white_medians = calculateb_median(white_w, column_name='volatile acidity', B=1000)

#Calulating median differences 
r_median_differences = white_medians - red_medians 

r_median_differences


array([-0.26  , -0.27  , -0.26  , -0.26  , -0.26  , -0.25  , -0.27  ,
       -0.26  , -0.26  , -0.26  , -0.26  , -0.25  , -0.25  , -0.25  ,
       -0.24  , -0.26  , -0.26  , -0.26  , -0.27  , -0.24  , -0.26  ,
       -0.26  , -0.26  , -0.25  , -0.27  , -0.26  , -0.25  , -0.26  ,
       -0.26  , -0.26  , -0.27  , -0.25  , -0.25  , -0.26  , -0.27  ,
       -0.27  , -0.26  , -0.26  , -0.26  , -0.27  , -0.26  , -0.27  ,
       -0.26  , -0.27  , -0.26  , -0.25  , -0.27  , -0.25  , -0.27  ,
       -0.24  , -0.26  , -0.255 , -0.24  , -0.25  , -0.26  , -0.26  ,
       -0.26  , -0.27  , -0.25  , -0.26  , -0.26  , -0.26  , -0.27  ,
       -0.27  , -0.26  , -0.26  , -0.28  , -0.26  , -0.25  , -0.26  ,
       -0.26  , -0.27  , -0.25  , -0.25  , -0.26  , -0.27  , -0.25  ,
       -0.28  , -0.26  , -0.26  , -0.25  , -0.26  , -0.27  , -0.26  ,
       -0.26  , -0.26  , -0.25  , -0.25  , -0.24  , -0.25  , -0.26  ,
       -0.25  , -0.26  , -0.27  , -0.26  , -0.27  , -0.26  , -0.26  ,
       -0.26  , -0.2

> Question 7

In [138]:
#Find the 0.05 quantiles of the 1000 values 90% bootstrap confidence interval of the median volatile acidity
lower_quantile = np.percentile(r_median_differences, 5)  
##Find the 0.95 quantiles of the 1000 values 90% bootstrap confidence interval of the median volatile acidity
upper_quantile = np.percentile(r_median_differences, 95)  

print(lower_quantile)
print(upper_quantile)

-0.27
-0.25


> Question 8

Since the confidence interval is entirely negative, we can tell white wines have significantly 
lower median volatile acidity than red wines.


> Question 9

In [139]:
# Reshape as 1 row, 1000 columns
rmd_reshaped = r_median_differences.reshape(1, -1)  

# Compute the bootstrap confidence interval for the median
result = bootstrap((rmd_reshaped), np.median, n_resamples=1000, confidence_level=0.90, axis=0)

result.confidence_interval

ConfidenceInterval(low=-0.26, high=-0.26)